Get data from github

In [1]:
!wget https://raw.githubusercontent.com/shitkov/bert4classification/main/bert_dataset.py

--2023-06-25 21:40:35--  https://raw.githubusercontent.com/shitkov/bert4classification/main/bert_dataset.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 891 [text/plain]
Saving to: ‘bert_dataset.py’

bert_dataset.py     100%[===================>]     891  --.-KB/s    in 0s      

2023-06-25 21:40:35 (51.0 MB/s) - ‘bert_dataset.py’ saved [891/891]



Installing Packages

In [2]:
!pip install sentencepiece==0.1.95

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 508.7/508.7 kB 35.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentencepiece: filename=sentencepiece-0.1.95-cp310-cp310-linux_x86_64.whl size=1546213 sha256=baff20ab3701cb56e8cc49e739316660c12634fef2a35cc424c88914a1f9fae8
  Stored in directory: /root/.cache/pip/wheels/ef/a4/01/5a500fc0c5a38917ef408c245eb40b7ac96f4a30fc6a346a4c
Successfully built sentencepiece


In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 68.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 103.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.7 MB/s eta 0:00:00


In [4]:
import pandas as pd

In [5]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 62.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13707 sha256=76a0cf544235f2b3c1bb7360f739d463b6a4dfa1d1f0736c043d8ad689b0a1bf
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [6]:
from bert_dataset import CustomDataset
from bert_classifier import BertClassifier

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
RANDOM_STATE = 12

In [9]:
BACKTEST_CCY = 'USDRUB'
BACKTEST_DAYS = 5
YEARS = [2021, 2022]

PATH_TEXTS = 'drive/MyDrive/data/telegram'
PATH_OPT_PNL = 'drive/MyDrive/data/pnl/option'

N_EPOCHS = 10

In [10]:
import os

# Get all available data_sources
sources = os.listdir(PATH_TEXTS)
sources

['bitkogan.csv',
 'War_Wealth_Wisdom.csv',
 'themovchans.csv',
 'vts.csv',
 'sky_bond.csv',
 'Alfa_Wealth.csv',
 'rshb_invest.csv',
 'headlines_QUANTS.csv',
 'signal.csv',
 'mmi.csv',
 'cbonds.csv']

In [11]:
!pip install fast_ml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 kB 5.2 MB/s eta 0:00:00


In [12]:
# Create target variable dataframe
pnl = pd.DataFrame()
for year in YEARS:
    pnl_year = pd.read_csv(f'{PATH_OPT_PNL}/Backtest_{BACKTEST_CCY}_{BACKTEST_DAYS}_days_{year}.txt')
    print(f'{PATH_OPT_PNL}/Backtest_{BACKTEST_CCY}_{BACKTEST_DAYS}_days_{year}.txt')
    pnl = pnl.append(pnl_year)
pnl['date_start'] = pd.to_datetime(pnl['date_start']).dt.strftime('%Y-%m-%d')
pnl.dropna(subset=['pnl'], inplace=True)

drive/MyDrive/data/pnl/option/Backtest_USDRUB_5_days_2021.txt


<ipython-input-12-fae68a3a6ca2>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pnl = pnl.append(pnl_year)


drive/MyDrive/data/pnl/option/Backtest_USDRUB_5_days_2022.txt


<ipython-input-12-fae68a3a6ca2>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pnl = pnl.append(pnl_year)


In [13]:
pnl = pnl.sort_values(by=['date_start'])

In [14]:
TRAIN_VAL_LEN = 0.8

test_idx = int(TRAIN_VAL_LEN * pnl.shape[0])

test_idx

314

In [16]:
import numpy as np

pnl_mean = pnl['pnl'][:test_idx].mean()
pnl_std = np.std(pnl['pnl'][:test_idx])

pnl['pnl'] = (pnl['pnl'] - pnl_mean) / pnl_std

pnl_std

1474845.2814802572

In [18]:
vol = np.std(pnl['pnl'])
vol

1.0322002766564826

In [19]:
pnl['date_start'].tolist()[0], pnl['date_start'].tolist()[test_idx]

('2021-01-04', '2022-06-16')

In [20]:
pnl['pnl_sign'] = pnl['pnl'].apply(lambda x: 2 if x >= 0.12 else (0 if x <= -0.12 else 1))
pnl

,date_start,pnl,pnl_sign
0,2021-01-04,0.233763,2
1,2021-01-05,-0.243072,0
2,2021-01-06,0.389866,2
3,2021-01-08,0.078885,1
4,2021-01-11,-0.328237,0
...,...,...,...
138,2022-09-28,-0.227259,0
139,2022-09-29,0.581439,2
140,2022-09-30,1.083376,2
141,2022-10-03,1.302921,2


In [21]:
pnl[pnl['pnl_sign'] == 2]['pnl_sign'].shape[0] / pnl.shape[0], pnl[pnl['pnl_sign'] == 1]['pnl_sign'].shape[0] / pnl.shape[0], pnl[pnl['pnl_sign'] == 0]['pnl_sign'].shape[0] / pnl.shape[0]

(0.3791348600508906, 0.23918575063613232, 0.3816793893129771)

In [22]:
import datetime as dt
from typing import List, Tuple, Union


def _binary_search_by_date(array: List[Tuple[dt.datetime, float]], date_x: dt.datetime) -> Union[int, None]:
    """
    Searches for the index of date_x in the array via binary search.

        Parameters:
            array (list) : A sorted array of (date, float_value) tuples
            date_x (datetime.datetime) : Date to search for

        Returns:
            index_x (int): Index of the searched date in the array.
    """
    left = 0
    right = len(array) - 1

    while left <= right:
        mid = left + (right - left) // 2

        if array[mid - 1][0] <= date_x <= array[mid][0]:
            return mid
        elif date_x > array[mid - 1][0] and date_x > array[mid][0]:
            left = mid + 1
        else:
            right = mid - 1

    return None


def binary_search_time_series(time_series: List[Tuple[dt.datetime, float]], date_start: dt.datetime,
                              date_end: dt.datetime) -> Union[List[Tuple[dt.datetime, float]], None]:
    """
    Searches for the part of the time series that is contained inside [date_start; date_end] period via binary search.

        Parameters:
            time_series (list) : A sorted array of (date, float_value) tuples
            date_start (datetime.datetime) : Starting date of the searched period
            date_end (datetime.datetime) : Ending date of the searched period

        Returns:
            time_series_data (list): Part of the time series that is contained inside [date_start; date_end] period.
    """

    if date_start <= date_end:
        left_index = _binary_search_by_date(time_series, date_start)

        if left_index is None:
            return None

        right_index = _binary_search_by_date(time_series[left_index:], date_end)

        if right_index is None:
            return None

        right_index += left_index
    else:
        left_index = _binary_search_by_date(time_series, date_end)

        if left_index is None:
            return None

        right_index = _binary_search_by_date(time_series[left_index:], date_start)

        if right_index is None:
            return None

        right_index += left_index

    return time_series[left_index:right_index]

In [23]:
pnl_sign_ts = [(pd.to_datetime(row['date_start']).to_pydatetime(), row['pnl_sign']) for  _, row in pnl.iterrows()]
pnl_sign_ts

[(datetime.datetime(2021, 1, 4, 0, 0), 2),
 (datetime.datetime(2021, 1, 5, 0, 0), 0),
 (datetime.datetime(2021, 1, 6, 0, 0), 2),
 (datetime.datetime(2021, 1, 8, 0, 0), 1),
 (datetime.datetime(2021, 1, 11, 0, 0), 0),
 (datetime.datetime(2021, 1, 12, 0, 0), 0),
 (datetime.datetime(2021, 1, 13, 0, 0), 2),
 (datetime.datetime(2021, 1, 14, 0, 0), 1),
 (datetime.datetime(2021, 1, 15, 0, 0), 1),
 (datetime.datetime(2021, 1, 18, 0, 0), 2),
 (datetime.datetime(2021, 1, 19, 0, 0), 2),
 (datetime.datetime(2021, 1, 20, 0, 0), 2),
 (datetime.datetime(2021, 1, 21, 0, 0), 2),
 (datetime.datetime(2021, 1, 22, 0, 0), 2),
 (datetime.datetime(2021, 1, 25, 0, 0), 2),
 (datetime.datetime(2021, 1, 26, 0, 0), 1),
 (datetime.datetime(2021, 1, 27, 0, 0), 2),
 (datetime.datetime(2021, 1, 28, 0, 0), 1),
 (datetime.datetime(2021, 1, 29, 0, 0), 2),
 (datetime.datetime(2021, 2, 1, 0, 0), 0),
 (datetime.datetime(2021, 2, 2, 0, 0), 0),
 (datetime.datetime(2021, 2, 3, 0, 0), 0),
 (datetime.datetime(2021, 2, 4, 0, 0), 

In [24]:
initial_date = pnl_sign_ts[0][0]

def get_pnl_sign(row):
    date_x = pd.to_datetime(row['date']).to_pydatetime()
    pnl_sign_key = _binary_search_by_date(pnl_sign_ts, date_x)

    if pnl_sign_key is not None and date_x >= initial_date:
        return pnl_sign_ts[_binary_search_by_date(pnl_sign_ts, date_x)][1]

In [25]:
# Generate dataframes
df = pd.DataFrame()

for s in sources:
    if s != '.gitignore':
        source_data = pd.read_csv(f'{PATH_TEXTS}/{s}')
        df = df.append(source_data)

df.head()

<ipython-input-25-fe866cb0e0b2>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(source_data)
<ipython-input-25-fe866cb0e0b2>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(source_data)
<ipython-input-25-fe866cb0e0b2>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(source_data)
<ipython-input-25-fe866cb0e0b2>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(source_data)
<ipython-input-25-fe866cb0e0b2>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(source_data)
<ipython-i

,id,date,text
0,3,2018-01-15T16:12:18,из российских биржевых идеек я бы посмотрел се...
1,5,2018-01-16T00:47:25,рекомендовал пару недель назад xmg. cn\nпервые...
2,7,2018-01-16T14:48:39,несмотря на страшно. ужас ужас. выкупаю паден...
3,8,2018-01-16T15:04:23,"январское ралли....в принципе, это не нечто не..."
4,10,2018-01-16T21:36:35,все идет по плану


In [26]:
df['pnl_sign'] = df.apply(lambda row: get_pnl_sign(row), axis=1)
df.dropna(subset=['pnl_sign'], inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()

,id,date,text,pnl_sign
0,10186,2021-01-05T09:37:44,Почему просели котировки нефти. Все очень прос...,2.0
1,10187,2021-01-05T09:41:54,Brent,2.0
2,10188,2021-01-05T09:42:23,Light,2.0
3,10189,2021-01-05T10:01:04,По итогам первого торгового дня года получил з...,2.0
4,10190,2021-01-05T10:09:56,Что-то непонятное происходит с рублем. USD/RUB...,2.0


In [27]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [28]:
import re
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords

from functools import lru_cache
from tqdm.notebook import tqdm
from tqdm.gui import tqdm as tqdm_gui

m = MorphAnalyzer()
regex = re.compile("[а-яa-zёЁ]+")

def words_only(text, regex=regex):
    try:
        return regex.findall(text.lower())
    except:
        return []

@lru_cache(maxsize=128)
def lemmatize_word(token, pymorphy=m):
    return pymorphy.parse(token)[0].normal_form

def lemmatize_text(text):
    return [lemmatize_word(w) for w in text]

mystopwords = stopwords.words('russian')
def remove_stopwords(lemmas, stopwords = mystopwords):
    return [w for w in lemmas if not w in stopwords and len(w) > 3]

def clean_text(text):
    tokens = words_only(text)
    lemmas = lemmatize_text(tokens)

    return ' '.join(remove_stopwords(lemmas))

In [29]:
lemmas = list(tqdm(map(clean_text, df['text']), total=len(df['text'].to_list())))

  0%|          | 0/45412 [00:00<?, ?it/s]

In [30]:
df['lemmas'] = lemmas
df = df[df['lemmas'] != '']
df.head()

,id,date,text,pnl_sign,lemmas
0,10186,2021-01-05T09:37:44,Почему просели котировки нефти. Все очень прос...,2.0,почему просесть котировка нефть очень просто п...
1,10187,2021-01-05T09:41:54,Brent,2.0,brent
2,10188,2021-01-05T09:42:23,Light,2.0,light
3,10189,2021-01-05T10:01:04,По итогам первого торгового дня года получил з...,2.0,итог первый торговый день получить звонок встр...
4,10190,2021-01-05T10:09:56,Что-то непонятное происходит с рублем. USD/RUB...,2.0,непонятный происходить рубль взлететь минута п...


In [31]:
df.shape

(45362, 5)

Initialize BERT classifier

In [32]:
classifier = BertClassifier(
        model_path='cointegrated/rubert-tiny',
        tokenizer_path='cointegrated/rubert-tiny',
        n_classes=len(np.unique(df['pnl_sign'])),
        epochs=N_EPOCHS,
        model_save_path='/content/bert.pt'
)

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny a

Prepare data and helpers for train and evlauation

In [33]:
train_valid = df[df['date'] <= pnl.iloc[test_idx]['date_start']]
test = df[df['date'] >= pnl.iloc[test_idx + 1]['date_start']]

In [34]:
from sklearn.model_selection import train_test_split

train, valid = train_test_split(train_valid, test_size=0.1, random_state=RANDOM_STATE)

In [35]:
train.shape, valid.shape, test.shape

((34226, 5), (3803, 5), (7211, 5))

In [36]:
classifier.preparation(
        X_train=train['lemmas'].tolist(),
        y_train=train['pnl_sign'].tolist(),
        X_valid=valid['lemmas'].tolist(),
        y_valid=valid['pnl_sign'].tolist()
    )

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Train loop

In [37]:
classifier.train()

Epoch 1/10
Train loss 1.0907290560229659 accuracy 0.44206159060363465
Val loss 1.0763121026202833 accuracy 0.46042597948987646
----------
Epoch 2/10
Train loss 1.0503218447884135 accuracy 0.4889557646233857
Val loss 1.0620229739057279 accuracy 0.4625295819090192
----------
Epoch 3/10
Train loss 1.0127741501222032 accuracy 0.5246303979430842
Val loss 1.1070644046002696 accuracy 0.455955824349198
----------
Epoch 4/10
Train loss 0.9794509028483108 accuracy 0.562291824928417
Val loss 1.2206458782495149 accuracy 0.47594004733105444
----------
Epoch 5/10
Train loss 0.9715919550722684 accuracy 0.598463156664524
Val loss 1.3903915108936584 accuracy 0.48146200368130426
----------
Epoch 6/10
Train loss 0.9704256534129557 accuracy 0.6283527143107579
Val loss 1.664537135494254 accuracy 0.48251380489087564
----------
Epoch 7/10
Train loss 0.9758039874753793 accuracy 0.6574533980015193
Val loss 1.9464783806817878 accuracy 0.48303970549566133
----------
Epoch 8/10
Train loss 0.9729544040721613 accur

Check test data

In [38]:
predictions = [classifier.predict(t) for t in test['lemmas'].tolist()]

In [39]:
from sklearn.metrics import precision_recall_fscore_support

precision, recall, f1score = precision_recall_fscore_support(test['pnl_sign'].tolist(), predictions, average='macro')[:3]

print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')

precision: 0.3202511404865608, recall: 0.32709305254098126, f1score: 0.3152243985207725


In [40]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test.tolist(), predictions)
cm

NameError: ignored

In [ ]:
y_test.shape